In [1]:
# Import necessary libraries
import os
import cv2
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
from skimage.morphology import skeletonize, remove_small_objects, binary_closing
import networkx as nx
import matplotlib.pyplot as plt
from patchify import patchify, unpatchify
import tensorflow.keras.backend as K

# Constants
patch_size = 256
model_filepath = "artjom_234535_unet_model_256px.h5"  # Update this with your actual model path
test_images_folder = "Kaggle"  # Directory containing test images
output_csv_file = "submission_4.csv"

# Define F1-score calculation
def compute_f1_score(y_actual, y_predicted):
    def calc_recall(y_actual, y_predicted):
        true_positive = K.sum(K.round(K.clip(y_actual * y_predicted, 0, 1)))
        total_actual_positives = K.sum(K.round(K.clip(y_actual, 0, 1)))
        return true_positive / (total_actual_positives + K.epsilon())

    def calc_precision(y_actual, y_predicted):
        true_positive = K.sum(K.round(K.clip(y_actual * y_predicted, 0, 1)))
        total_predicted_positives = K.sum(K.round(K.clip(y_predicted, 0, 1)))
        return true_positive / (total_predicted_positives + K.epsilon())

    precision = calc_precision(y_actual, y_predicted)
    recall = calc_recall(y_actual, y_predicted)
    return 2 * ((precision * recall) / (precision + recall + K.epsilon()))

# Load the trained model
model = load_model(model_filepath, custom_objects={"f1": compute_f1_score})

# Helper functions
def full_remove_black_borders(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, binary = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        raise ValueError("No contours found.")
    largest_contour = max(contours, key=cv2.contourArea)
    x, y, w, h = cv2.boundingRect(largest_contour)
    cropped_image = image[y:y + h, x:x + w]
    return cropped_image

def clean_mask(mask):
    mask = remove_small_objects(mask > 0, min_size=200)
    mask = binary_closing(mask, footprint=np.ones((5, 5)))
    return (mask > 0).astype(np.uint8)

def segment_roots(image, model):
    h, w, _ = image.shape
    pad_h = (patch_size - h % patch_size) % patch_size
    pad_w = (patch_size - w % patch_size) % patch_size
    padded_image = np.pad(image, ((0, pad_h), (0, pad_w), (0, 0)), mode='constant')
    dish_patches = patchify(padded_image, (patch_size, patch_size, 3), step=patch_size)
    predicted_patches = []
    for i in range(dish_patches.shape[0]):
        for j in range(dish_patches.shape[1]):
            patch = dish_patches[i, j, 0]
            patch = cv2.cvtColor(patch, cv2.COLOR_BGR2GRAY)
            patch = patch / 255.0
            patch = np.expand_dims(patch, axis=(0, -1))
            prediction = model.predict(patch)
            predicted_patches.append(prediction[0, :, :, 0])
    predicted_patches = np.array(predicted_patches)
    predicted_patches = predicted_patches.reshape(dish_patches.shape[0], dish_patches.shape[1], patch_size, patch_size)
    predicted_mask = unpatchify(predicted_patches, padded_image.shape[:2])
    cleaned_mask = clean_mask((predicted_mask > 0.5).astype(np.uint8))
    return cleaned_mask[:h, :w]

def extract_individual_roots(mask, image_width):
    plant_width = image_width // 5
    individual_roots = []
    for i in range(5):
        x_start = i * plant_width
        x_end = (i + 1) * plant_width
        plant_mask = mask[:, x_start:x_end]
        individual_roots.append(plant_mask)
    return individual_roots

def extract_primary_root(mask):
    skeleton = skeletonize(mask > 0)
    coords = np.column_stack(np.nonzero(skeleton))
    if len(coords) == 0:
        return 0
    G = nx.Graph()
    coord_to_node = {tuple(coord): idx for idx, coord in enumerate(coords)}
    for coord in coords:
        y, x = coord
        G.add_node(coord_to_node[tuple(coord)], coord=(y, x))
        for dy, dx in [(-1, 0), (1, 0), (0, -1), (0, 1), (-1, -1), (-1, 1), (1, -1), (1, 1)]:
            neighbor = (y + dy, x + dx)
            if neighbor in coord_to_node:
                G.add_edge(coord_to_node[tuple(coord)], coord_to_node[neighbor])
    start_node = min(G.nodes, key=lambda n: G.nodes[n]["coord"][0])
    lengths, paths = nx.single_source_dijkstra(G, start_node)
    end_node = max(lengths, key=lengths.get)
    return lengths[end_node]

def locate_and_crop_dish(image_path):
    grayscale_image = cv2.imread(image_path, 0)
    original_image = cv2.imread(image_path)
    blurred_image = cv2.medianBlur(grayscale_image, 5)
    _, binary_threshold = cv2.threshold(blurred_image, 150, 255, cv2.THRESH_BINARY)
    morph_kernel = np.ones((9, 9), np.uint8)
    morph_closed = cv2.erode(cv2.dilate(binary_threshold, morph_kernel, iterations=3), morph_kernel, iterations=4)
    contours, _ = cv2.findContours(morph_closed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    largest_shape = max(contours, key=cv2.contourArea)
    x, y, width, height = cv2.boundingRect(largest_shape)
    bounding_size = max(width, height)
    center_x, center_y = x + width // 2, y + height // 2
    x1, y1 = center_x - bounding_size // 2, center_y - bounding_size // 2
    x2, y2 = x1 + bounding_size, y1 + bounding_size
    cropped_region = original_image[max(0, y1):min(grayscale_image.shape[0], y2),
                                    max(0, x1):min(grayscale_image.shape[1], x2)]
    return cropped_region, (max(0, x1), max(0, y1), min(grayscale_image.shape[1], x2), min(grayscale_image.shape[0], y2))

def map_mask_to_original(predicted_mask, original_image, crop_coordinates):
    original_height, original_width = original_image.shape[:2]
    aligned_mask = np.zeros((original_height, original_width), dtype=np.float32)
    x_start, y_start, x_end, y_end = crop_coordinates
    padding = 100
    x_end += padding
    y_end += padding
    cropped_height, cropped_width = y_end - y_start, x_end - x_start
    resized_prediction = cv2.resize(predicted_mask, (cropped_width, cropped_height), interpolation=cv2.INTER_LINEAR)
    aligned_mask[y_start:y_end, x_start:x_end] = resized_prediction
    return aligned_mask

# Pipeline
predictions = []
for image_name in sorted(os.listdir(test_images_folder)):
    if image_name.endswith(".png"):
        image_path = os.path.join(test_images_folder, image_name)
        original_image = cv2.imread(image_path)
        print(f"Processing {image_name}...")

        # Step 1: Detect and crop Petri dish
        cropped_image, crop_coords = locate_and_crop_dish(image_path)

        # Step 2: Segment roots
        root_mask = segment_roots(cropped_image, model)
        aligned_mask = map_mask_to_original(root_mask, original_image, crop_coords)

        # Step 3: Extract individual roots
        individual_roots = extract_individual_roots(aligned_mask, original_image.shape[1])

        # Step 4: Calculate primary root lengths
        for plant_idx, plant_mask in enumerate(individual_roots, start=1):
            primary_root_length = extract_primary_root(plant_mask)
            predictions.append({
                "Plant ID": f"{image_name.split('.')[0]}_plant_{plant_idx}",
                "Length (px)": primary_root_length
            })
            print(f"Plant {plant_idx}: Primary Root Length = {primary_root_length}")

# Save predictions to CSV
predictions_df = pd.DataFrame(predictions)
predictions_df.to_csv(output_csv_file, index=False)
print(f"Predictions saved to {output_csv_file}")

Processing test_image_1.png...
1/1 [==============================] - 0s 88ms/step
Plant 1: Primary Root Length = 0
Plant 2: Primary Root Length = 530
Plant 3: Primary Root Length = 252
Plant 4: Primary Root Length = 0
Plant 5: Primary Root Length = 0
Processing test_image_10.png...
1/1 [==============================] - 0s 114ms/step
Plant 1: Primary Root Length = 0
Plant 2: Primary Root Length = 279
Plant 3: Primary Root Length = 234
Plant 4: Primary Root Length = 1264
Plant 5: Primary Root Length = 256
Processing test_image_11.png...
1/1 [==============================] - 0s 94ms/step
Plant 1: Primary Root Length = 160
Plant 2: Primary Root Length = 74
Plant 3: Primary Root Length = 1412
Plant 4: Primary Root Length = 1499
Plant 5: Primary Root Length = 71
Processing test_image_12.png...
1/1 [==============================] - 0s 94ms/step
Plant 1: Primary Root Length = 146
Plant 2: Primary Root Length = 90
Plant 3: Primary Root Length = 908
Plant 4: Primary Root Length = 956
Plant 5

In [1]:
import os
import cv2
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
from skimage.morphology import skeletonize, remove_small_objects, binary_closing
import networkx as nx
import matplotlib.pyplot as plt
from patchify import patchify, unpatchify
import tensorflow.keras.backend as K

# Constants
patch_size = 256
model_filepath = "artjom_234535_unet_model_256px.h5"  # Update this with your actual model path
test_images_folder = "Kaggle"  # Directory containing test images
output_csv_file = "submission_5.csv"

# Define F1-score calculation
def compute_f1_score(y_actual, y_predicted):
    def calc_recall(y_actual, y_predicted):
        true_positive = K.sum(K.round(K.clip(y_actual * y_predicted, 0, 1)))
        total_actual_positives = K.sum(K.round(K.clip(y_actual, 0, 1)))
        return true_positive / (total_actual_positives + K.epsilon())

    def calc_precision(y_actual, y_predicted):
        true_positive = K.sum(K.round(K.clip(y_actual * y_predicted, 0, 1)))
        total_predicted_positives = K.sum(K.round(K.clip(y_predicted, 0, 1)))
        return true_positive / (total_predicted_positives + K.epsilon())

    precision = calc_precision(y_actual, y_predicted)
    recall = calc_recall(y_actual, y_predicted)
    return 2 * ((precision * recall) / (precision + recall + K.epsilon()))

# Load the trained model
model = load_model(model_filepath, custom_objects={"f1": compute_f1_score})

# Helper functions
def full_remove_black_borders(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, binary = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        raise ValueError("No contours found.")
    largest_contour = max(contours, key=cv2.contourArea)
    x, y, w, h = cv2.boundingRect(largest_contour)
    cropped_image = image[y:y + h, x:x + w]
    return cropped_image

def clean_mask(mask):
    mask = remove_small_objects(mask > 0, min_size=200)
    mask = binary_closing(mask, footprint=np.ones((5, 5)))
    return (mask > 0).astype(np.uint8)

def segment_roots(image, model):
    h, w, _ = image.shape
    pad_h = (patch_size - h % patch_size) % patch_size
    pad_w = (patch_size - w % patch_size) % patch_size
    padded_image = np.pad(image, ((0, pad_h), (0, pad_w), (0, 0)), mode='constant')
    dish_patches = patchify(padded_image, (patch_size, patch_size, 3), step=patch_size)
    predicted_patches = []
    for i in range(dish_patches.shape[0]):
        for j in range(dish_patches.shape[1]):
            patch = dish_patches[i, j, 0]
            patch = cv2.cvtColor(patch, cv2.COLOR_BGR2GRAY)
            patch = patch / 255.0
            patch = np.expand_dims(patch, axis=(0, -1))
            prediction = model.predict(patch)
            predicted_patches.append(prediction[0, :, :, 0])
    predicted_patches = np.array(predicted_patches)
    predicted_patches = predicted_patches.reshape(dish_patches.shape[0], dish_patches.shape[1], patch_size, patch_size)
    predicted_mask = unpatchify(predicted_patches, padded_image.shape[:2])
    cleaned_mask = clean_mask((predicted_mask > 0.5).astype(np.uint8))
    return cleaned_mask[:h, :w]

def extract_individual_roots(mask, image_width):
    plant_width = image_width // 5
    individual_roots = []
    for i in range(5):
        x_start = i * plant_width
        x_end = (i + 1) * plant_width
        plant_mask = mask[:, x_start:x_end]
        individual_roots.append(plant_mask)
    return individual_roots

def extract_primary_root(mask):
    skeleton = skeletonize(mask > 0)
    coords = np.column_stack(np.nonzero(skeleton))
    if len(coords) == 0:
        return 0
    G = nx.Graph()
    coord_to_node = {tuple(coord): idx for idx, coord in enumerate(coords)}
    for coord in coords:
        y, x = coord
        G.add_node(coord_to_node[tuple(coord)], coord=(y, x))
        for dy, dx in [(-1, 0), (1, 0), (0, -1), (0, 1), (-1, -1), (-1, 1), (1, -1), (1, 1)]:
            neighbor = (y + dy, x + dx)
            if neighbor in coord_to_node:
                G.add_edge(coord_to_node[tuple(coord)], coord_to_node[neighbor])
    start_node = min(G.nodes, key=lambda n: G.nodes[n]["coord"][0])
    lengths, paths = nx.single_source_dijkstra(G, start_node)
    end_node = max(lengths, key=lengths.get)
    return lengths[end_node]

# Pipeline
predictions = []
for image_name in sorted(os.listdir(test_images_folder)):
    if image_name.endswith(".png"):
        image_path = os.path.join(test_images_folder, image_name)
        original_image = cv2.imread(image_path)
        print(f"Processing {image_name}...")

        # Step 1: Detect and crop Petri dish
        cropped_image = full_remove_black_borders(original_image)

        # Step 2: Segment roots
        root_mask = segment_roots(cropped_image, model)

        # Step 3: Extract individual roots
        individual_roots = extract_individual_roots(root_mask, cropped_image.shape[1])

        # Step 4: Calculate primary root lengths
        for plant_idx, plant_mask in enumerate(individual_roots, start=1):
            primary_root_length = extract_primary_root(plant_mask)
            predictions.append({
                "Plant ID": f"{image_name.split('.')[0]}_plant_{plant_idx}",
                "Length (px)": primary_root_length
            })
            print(f"Plant {plant_idx}: Primary Root Length = {primary_root_length}")

# Save predictions to CSV
predictions_df = pd.DataFrame(predictions)
predictions_df.to_csv(output_csv_file, index=False)
print(f"Predictions saved to {output_csv_file}")

Processing test_image_1.png...
1/1 [==============================] - 0s 91ms/step
Plant 1: Primary Root Length = 515
Plant 2: Primary Root Length = 0
Plant 3: Primary Root Length = 243
Plant 4: Primary Root Length = 0
Plant 5: Primary Root Length = 0
Processing test_image_10.png...
1/1 [==============================] - 0s 90ms/step
Plant 1: Primary Root Length = 268
Plant 2: Primary Root Length = 1504
Plant 3: Primary Root Length = 225
Plant 4: Primary Root Length = 1214
Plant 5: Primary Root Length = 1045
Processing test_image_11.png...
1/1 [==============================] - 0s 86ms/step
Plant 1: Primary Root Length = 763
Plant 2: Primary Root Length = 69
Plant 3: Primary Root Length = 1360
Plant 4: Primary Root Length = 1447
Plant 5: Primary Root Length = 883
Processing test_image_12.png...
1/1 [==============================] - 0s 85ms/step
Plant 1: Primary Root Length = 94
Plant 2: Primary Root Length = 1119
Plant 3: Primary Root Length = 873
Plant 4: Primary Root Length = 922
Pl